<a href="https://colab.research.google.com/github/marcin192/corona/blob/master/Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plt.rcParams['figure.figsize'] = [15, 5]
from IPython import display
from ipywidgets import interact, widgets
pd.set_option('mode.chained_assignment', None)

In [0]:
def download(option):
  if option=="Cases":
    url = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
  elif option=="Population":
    url=f"https://raw.githubusercontent.com/marcin192/pop/master/popek.csv"
  dfC = pd.read_csv(url, error_bad_lines=False)
  return dfC

In [0]:
def differentiate(datex,dt):
  datexC=datex.strftime('%m/%d/%y').lstrip("0").replace(" 0", " ").replace(" 0/"," /")
  dateo=datex+datetime.timedelta(days=-dt)
  dateoC=dateo.strftime('%m/%d/%y').lstrip("0").replace(" 0", " ").replace(" 0/"," /")
  dfC=download("Cases")
  dfM=dfC[["Country/Region",datexC,dateoC]]
  dfM[["diff"]]=-dfM[[dateoC]]
  dfM[["diff"]]=0
  it=0
  l=dfM.shape[0]
  for i in range(0,l):
    dfM["diff"].iloc[it:it+1]=(dfM[datexC].iloc[it:it+1]-dfM[dateoC].iloc[it:it+1])/dt
    it=it+1
  
  return dfM.sort_values(by="diff",ascending=False)


In [0]:
def plotd(date,dt,ex):
  """formated_gdf = dfD.groupby(["Country/Region"]).max()"""
  dfg=differentiate(date,dt)
  dfg=dfg.groupby(["Country/Region"]).sum()
  formated_gdf = dfg.reset_index()
  formated_gdf['size'] = formated_gdf[["diff"]].pow(ex)



  fig = px.choropleth(formated_gdf, locations="Country/Region", locationmode='country names', 
                     color="diff", hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf["diff"].pow(ex))+2], 
                     projection="natural earth",
                     labels={"diff":"Średni dzienny przyrost"},
                     title=f"Średni dzienny przyrost w okresie od {date+datetime.timedelta(days=-dt)} do {date}")





  fig.show()
  return formated_gdf.sort_values(by="diff",ascending=False)

In [0]:
def population(option):
  pop=download("Population")
  pop=pop.sort_values(by="Country")
  pop=pop.reset_index()
  pop=pop.drop(["index"],axis=1)
  if(option==1):
    pop=pop.drop(["Country"],axis=1)
    return pop
  else:
    return pop

In [0]:
def confirmparse():
  dfC=download("Cases")
  dfC=dfC.groupby(by="Country/Region").sum().reset_index()
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="Kosovo"].index,axis=0)
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="Diamond Princess"].index,axis=0)
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="West Bank and Gaza"].index,axis=0)
  dfC=dfC.reset_index()
  dfC=dfC.drop(["index"],axis=1)
  index1=dfC[dfC["Country/Region"]=="Congo (Brazzaville)"].index
  index2=dfC[dfC["Country/Region"]=="Congo (Kinshasa)"].index
  dfC.loc[index1[0]] += dfC.loc[index2[0]]
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="Congo (Kinshasa)"].index,axis=0)
  dfC.at[index1[0],"Country/Region"]="Congo"
  dfC=dfC.drop(["Lat","Long"],axis=1)
  dfC=dfC.reset_index()
  dfC=dfC.drop(["index"],axis=1)
  return dfC

In [0]:
def differentiate_population(datex,dt):
  datexC=datex.strftime('%m/%d/%y').lstrip("0").replace(" 0", " ").replace(" 0/"," /")
  dateo=datex+datetime.timedelta(days=-dt)
  dateoC=dateo.strftime('%m/%d/%y').lstrip("0").replace(" 0", " ").replace(" 0/"," /")
  c=confirmparse()
  p=population(1)
  j=pd.concat([p,c],axis=1)
  m=j[["Population","Country/Region",datexC,dateoC]]
  m[["diff%"]]=m[[dateoC]]
  m[["diff%"]]=0
  it=0
  l=m.shape[0]
  for i in range(0,l):
    m["diff%"].iloc[it:it+1]=(m[datexC].iloc[it:it+1]-m[dateoC].iloc[it:it+1])/dt/(m["Population"].iloc[it:it+1])*1000000
    it=it+1
  
  return m.sort_values(by="diff%",ascending=False)

In [0]:
def plotdp(date,dt,ex):
  """formated_gdf = dfD.groupby(["Country/Region"]).max()"""
  dfg=differentiate_population(date,dt)
  dfg=dfg.groupby(["Country/Region"]).sum()
  indexNames=dfg[dfg["Population"]<1000000].index
  dfg.drop(indexNames , inplace=True)
  formated_gdf = dfg.reset_index()
  formated_gdf['size'] = formated_gdf[["diff%"]].pow(ex)



  fig = px.choropleth(formated_gdf, locations="Country/Region", locationmode='country names', 
                     color="diff%", hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf["diff%"].pow(ex))+2], 
                     projection="natural earth",
                     labels={"diff":"Średni dzienny przyrost"},
                     title=f"Średni dzienny przyrost na 1 000 000 mieszkańców w okresie od {date+datetime.timedelta(days=-dt)} do {date}")





  fig.show()
  return formated_gdf.sort_values(by="diff%",ascending=False)

ERROR! Session/line number was not unique in database. History logging moved to new session 60
